In [1]:
import pandas as pd
import psycopg2

In [2]:
try: 
    conn=psycopg2.connect(
       host='sumar.larioja.gob.ar',
       user='postgres',
       password='sumarLR21',
       database='sumar'
       )
    
    print("conexion exitosa")
    cursor = conn.cursor()
    cursor.execute("SELECT version()")
    row=cursor.fetchone()
    print(row)
    
except Exception as ex:
    print(ex)

conexion exitosa
('PostgreSQL 9.0.1, compiled by Visual C++ build 1500, 32-bit',)


In [3]:
# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Consulta SQL para obtener la lista de esquemas
query = "SELECT schema_name FROM information_schema.schemata"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
esquemas = cursor.fetchall()

# Imprimir los esquemas
print("Lista de esquemas:")
for esquema in esquemas:
    print(esquema[0])

# Cerrar el cursor y la conexión
cursor.close()

Lista de esquemas:
pg_toast
pg_temp_1
pg_toast_temp_1
pg_catalog
information_schema
alta
calidad
capacitacion
cardiopatia
centro
contabilidad
expediente
facturacion
fichero
general
leche
mensajes
monitoreo
nacer
nomenclador
permisos
personal
planillas
pss
public
puco
sistema
tareas_divisionsoft
trazadoras
uad
laravel


In [4]:
# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Esquema específico que deseas consultar
esquema_especifico = "monitoreo"

# Consulta SQL para obtener las tablas en un esquema específico
query = f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{esquema_especifico}'"

# Ejecutar la consulta
cursor.execute(query)

# Obtener los resultados
tablas = cursor.fetchall()

# Imprimir las tablas
print(f"Tablas en el esquema '{esquema_especifico}':")
for tabla in tablas:
    print(tabla[0])

# Cerrar el cursor y la conexión
cursor.close()

Tablas en el esquema 'monitoreo':
totales_metas_trazadoras
metas_ceb
totales_metas_varias
ceb_anual
poblacionobjetivo
totales_ceb
ceb_mensual


In [5]:
# Consulta SQL para obtener los datos de una tabla en un esquema específico
query = """
SELECT                  o.cuie,
                        p."month" AS Periodo,
                        o.total AS poblacion_objetivo,
                        p.total AS poblacion_CEB,
			o."a" AS "grupo 0-5",
			o.b AS "grupo 6-9",
			o."c" AS "grupo 10-19", 
			o.d AS "grupo 20-64 Mujer",
			o.e AS "grupo 20-64 h",
			o.f AS "grupo Mayores 64"
FROM "monitoreo"."poblacionobjetivo" AS o
LEFT JOIN "monitoreo"."ceb_mensual" AS p 
ON o.cuie = p.cuie
WHERE p."month" = (SELECT MAX("month") FROM "monitoreo"."ceb_mensual")
"""
# Crear un DataFrame a partir de la consulta
df_ceb = pd.read_sql_query(query, conn)

# Imprimir el DataFrame
print(df_ceb)

# Cerrar la conexión
cursor.close()
conn.close()

       cuie  periodo  poblacion_objetivo  poblacion_ceb  grupo 0-5  grupo 6-9  \
0    L00001       12                48.0           32.0        0.0        4.0   
1    L00002       12               154.0           67.0        1.0        0.0   
2    L00003       12               276.0          124.0      120.0       19.0   
3    L00005       12                54.0          146.0       29.0        2.0   
4    L00006       12                 0.0            0.0        0.0        0.0   
..      ...      ...                 ...            ...        ...        ...   
191  L10060       12              1128.0          522.0       74.0      126.0   
192  L94905       12               176.0           80.0       11.0        7.0   
193  L94909       12              1616.0          785.0      203.0      151.0   
194  L94914       12               126.0           63.0       15.0       19.0   
195  L94917       12                74.0           29.0        0.0        1.0   

     grupo 10-19  grupo 20-

C:\Users\FRANCISCO\AppData\Local\Temp\ipykernel_15172\8015524.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ceb = pd.read_sql_query(query, conn)


In [6]:
df_ceb

,cuie,periodo,poblacion_objetivo,poblacion_ceb,grupo 0-5,grupo 6-9,grupo 10-19,grupo 20-64 Mujer,grupo 20-64 h,grupo Mayores 64
0,L00001,12,48.0,32.0,0.0,4.0,14.0,20.0,10.0,0.0
1,L00002,12,154.0,67.0,1.0,0.0,0.0,0.0,152.0,1.0
2,L00003,12,276.0,124.0,120.0,19.0,39.0,55.0,41.0,2.0
3,L00005,12,54.0,146.0,29.0,2.0,7.0,12.0,4.0,0.0
4,L00006,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
191,L10060,12,1128.0,522.0,74.0,126.0,322.0,391.0,212.0,3.0
192,L94905,12,176.0,80.0,11.0,7.0,29.0,48.0,80.0,1.0
193,L94909,12,1616.0,785.0,203.0,151.0,275.0,529.0,438.0,20.0
194,L94914,12,126.0,63.0,15.0,19.0,38.0,40.0,14.0,0.0


In [7]:
df_ceb['poblacion_objetivo'] = df_ceb['poblacion_objetivo'].astype(int)
df_ceb['poblacion_ceb'] = df_ceb['poblacion_ceb'].astype(int)
df_ceb['grupo 0-5'] = df_ceb['grupo 0-5'].astype(int)
df_ceb['grupo 6-9'] = df_ceb['grupo 6-9'].astype(int)
df_ceb['grupo 10-19'] = df_ceb['grupo 10-19'].astype(int)
df_ceb['grupo 20-64 Mujer'] = df_ceb['grupo 20-64 Mujer'].astype(int)
df_ceb['grupo 20-64 h'] = df_ceb['grupo 20-64 h'].astype(int)
df_ceb['grupo Mayores 64'] = df_ceb['grupo Mayores 64'].astype(int)

In [8]:
df_ceb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   cuie                196 non-null    object
 1   periodo             196 non-null    int64 
 2   poblacion_objetivo  196 non-null    int32 
 3   poblacion_ceb       196 non-null    int32 
 4   grupo 0-5           196 non-null    int32 
 5   grupo 6-9           196 non-null    int32 
 6   grupo 10-19         196 non-null    int32 
 7   grupo 20-64 Mujer   196 non-null    int32 
 8   grupo 20-64 h       196 non-null    int32 
 9   grupo Mayores 64    196 non-null    int32 
dtypes: int32(8), int64(1), object(1)
memory usage: 9.3+ KB


In [9]:
df_ceb.to_csv('tablas_powerBI/df_ceb.csv', index=False)